# Battle Royale Environment Trainer
This notebook is for training Battle Royale agents. MADDPG is used for training the agents.

## Setup Environment Dependencies

In [1]:
import sys
from gym_unity.envs import UnityEnv

print("Python version:")
print(sys.version)
print(sys.executable)

# check Python version
if (sys.version_info[0] < 3):
    raise Exception("ERROR: ML-Agents Toolkit (v0.3 onwards) requires Python 3")

Python version:
3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
/Users/adhipradhana/anaconda3/envs/unity-battle-royale/bin/python


## Start Environment

In [54]:
# Environment name
# Remember to put battle royale environment configuration within the config folder
env_name = "environment/battle-royale"

env = UnityEnv(env_name, worker_id=0, use_visual=False, multiagent=True)

print(str(env))

INFO:mlagents_envs:Connected new brain:
PlayerBrain?team=0
INFO:gym_unity:2 agents within environment.


<UnityEnv instance>


## Testing Model

### Setup Algorithm Dependencies

In [55]:
import torch
import numpy as np
import random
import os
import visdom
from datetime import datetime

from utils.MADDPG import MADDPG

### Model Variables

In [56]:
random_seed = 1000
n_states = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]
n_agents = env.number_agents
n_episode = 100
max_steps = 100
buffer_capacity = 1000000
batch_size = 1000
episodes_before_train = 100

In [57]:
# setup seed
torch.manual_seed(random_seed)
np.random.seed(random_seed)

maddpg = MADDPG(n_agents, n_states, n_actions, batch_size, buffer_capacity, episodes_before_train, use_approx=True)

FloatTensor = torch.cuda.FloatTensor if maddpg.use_cuda else torch.FloatTensor

## Loading Model

In [58]:
## Model from experiment 4x4
# path_normal = os.path.join(os.getcwd(), 'checkpoint', '4x4 Normal', 'Time_2020-04-20_07:09:21.273684_NAgent_2_Episode_5000.pth')
# path_static = os.path.join(os.getcwd(), 'checkpoint', '4x4 Static', 'Time_2020-04-20_07:06:38.159288_NAgent_2_Episode_5000.pth')
# path_aggressive = os.path.join(os.getcwd(), 'checkpoint', '4x4 Aggressive', 'Time_2020-04-22_00:57:16.521466_NAgent_2_Episode_5000.pth')
path_passive = os.path.join(os.getcwd(), 'checkpoint', '4x4 Passive', 'Time_2020-04-21_02:53:27.519503_NAgent_2_Episode_5000.pth')

## Model from experiment 8x10
# path_aggressive = os.path.join(os.getcwd(), 'checkpoint', '8x10 Aggressive', 'Time_2020-04-24_10:41:51.445399_NAgent_2_Episode_5000.pth')
path_passive_new = os.path.join(os.getcwd(), 'checkpoint', '8x10 Passive', 'Time_2020-04-23_04:04:35.951300_NAgent_2_Episode_5000.pth')

## Model from experiment 6x6
# path_aggressive_new = os.path.join(os.getcwd(), 'checkpoint', '6x6 Aggressive', 'Time_2020-04-26_05:30:22.037620_NAgent_2_Episode_5000.pth')

## Model from new experiment 
# path_normal_new = os.path.join(os.getcwd(), 'checkpoint', 'New Environment', '6x6 Normal', 'Time_2020-04-29_04:43:47.774365_NAgent_2_Episode_5000.pth')

# maddpg.load(path=path_normal, map_location='cpu')
maddpg.load_agent(path=path_passive, agent_number=0, model_number=1, map_location='cpu')
maddpg.load_agent(path=path_passive_new, agent_number=1, model_number=1, map_location='cpu')
# maddpg.load_agent(path=path_aggressive, agent_number=0, model_number=0, map_location='cpu')
# maddpg.load_agent(path=path_passive, agent_number=1, model_number=1, map_location='cpu')

for i in range(maddpg.n_agents):
    print("Var {}: {}".format(i, maddpg.var[i]))

Var 0: 0.5591438069865847
Var 1: 0.4028315037868575


## Run Model

In [59]:
print("Testing model...")
for i_episode in range(n_episode):
    # reset environment
    obs = env.reset()
    obs = np.stack(obs)
    
    # convert observation to tensor
    if isinstance(obs, np.ndarray):
        obs = torch.from_numpy(obs).float()
    
    total_reward = 0.0
    rr = np.zeros((n_agents,))
    for i_step in range(max_steps):
        obs = obs.type(FloatTensor)
        actions = maddpg.select_action(obs).data.cpu()
        
        # for debug
#         actions[1] = torch.zeros(n_actions)
#         actions[1][4] = -1
        
        actions_list = actions.tolist()
        
        obs_, reward, done, _ = env.step(actions_list)
        
        reward = torch.FloatTensor(reward).type(FloatTensor)
        obs_ = np.stack(obs_)
        obs_ = torch.from_numpy(obs_).float()
        if i_step != max_steps - 1:
            next_obs = obs_
        else:
            next_obs = None

        total_reward += reward.sum()   
        rr += reward.cpu().numpy()
        obs = next_obs

        # check if done
        if True in done:
            print(done)
            break

    maddpg.episode_done += 1
    print("Episode: {}, reward = {}".format(i_episode, total_reward))
  
    if maddpg.episode_done == maddpg.episodes_before_train:
        print("Training begins...")
        print("MADDPG on Battle Royale")
    

Testing model...
Episode: 0, reward = -0.07499996572732925
[True, True]
Episode: 1, reward = 2.2750000953674316
[True, True]
Episode: 2, reward = 2.1575000286102295
[True, True]
Episode: 3, reward = 2.1649999618530273
Episode: 4, reward = -0.1799999475479126
[True, True]
Episode: 5, reward = 2.819999933242798
Episode: 6, reward = -0.38499999046325684
[True, True]
Episode: 7, reward = 2.0174999237060547
[True, True]
Episode: 8, reward = 2.1649999618530273
Episode: 9, reward = -0.1799999624490738
[True, True]
Episode: 10, reward = 1.902500033378601
[True, True]
Episode: 11, reward = 2.2100000381469727
[True, True]
Episode: 12, reward = 2.0975000858306885
Episode: 13, reward = -0.3199998736381531
Episode: 14, reward = -0.14499995112419128
[True, True]
Episode: 15, reward = 2.202500104904175
Episode: 16, reward = -0.2074998915195465
[True, True]
Episode: 17, reward = 2.2225000858306885
[True, True]
Episode: 18, reward = 1.8750001192092896
[True, True]
Episode: 19, reward = 2.21749997138977

## Close Environment

In [60]:
env.close()

INFO:mlagents_envs:Environment shut down with return code 0.
